In [1]:
# Limpar os dados
# Criar o node
# Criar a PPC
# NLIST 1 ITEMSET
# NLIST 2 ITEMSET

In [2]:
import itertools
import copy

In [3]:
support = 900
F = []
input_file = "pumsb.txt"

In [4]:
def make_itemset_1(filename):
    global support
    itemset_1 = {}
    with open(filename) as finput:
        f = finput.read().split('\n')
        for line in f:
            items = line.strip().split(' ')
            for item in items:
                if item not in itemset_1:
                    itemset_1[item] = 0
                itemset_1[item] += 1
    
    for k, v in list(itemset_1.items()):
        if v < support:
            del itemset_1[k]
    return itemset_1


itemset_1 = make_itemset_1(input_file)

In [5]:
class PPCNode:
    def __init__(self):
        self.pre_order = None
        self.post_order = None
        self.count = None
        self.label = None
        self.parent = None
        self.child = None
        self.sibling = None


class FPTNode:
    def __init__(self):
        self.equivalent_items = None
        self.child_nodes = None
        self.label = None
        self.itemset = None
        self.support = None
    
    def __repr__(self):
        return str(self.__dict__)

In [6]:
def create_node(root, label):
    node = PPCNode()
    node.label = label
    node.count = 1
    node.parent = root
    return node


def insert_ppc_node(root, item):
    if root.child is None:
        node = create_node(root, item)
        root.child = node
        return root.child
    elif root.child.label == item:
        root.child.count += 1
        return root.child
    elif root.child.sibling is None:
        node = create_node(root, item)
        root.child.sibling = node
        return root.child.sibling
    else:
        current_sibling = root.child.sibling
        last_sibling = None
        while current_sibling is not None:
            if current_sibling.label == item:
                current_sibling.count += 1
                return current_sibling
            else:
                last_sibling = current_sibling
                current_sibling = current_sibling.sibling
        node = create_node(root, item)
        last_sibling.sibling = node        
        return node

In [7]:
def pre_post(root, pre=0, post=0):
    root.pre_order = pre
    post_ = post
    if root.child is not None:
        root.child.pre_order = pre + 1
        pre, post_ = pre_post(root.child, root.child.pre_order, post)
    root.post_order = post_
    if root.sibling is not None:
        root.sibling.pre_order = pre + 1
        pre, post_ = pre_post(root.sibling, root.sibling.pre_order, root.post_order + 1)
        return pre, post_
    return pre, post_ + 1    

In [8]:
def build_ppc_tree(filename, itemset_1):
    root = PPCNode()
    with open(filename) as finput:
        f = finput.read().split('\n')
        for line in f:
            items = line.strip().split(' ')            
            transaction = []
            for item in items:
                if item in list(itemset_1.keys()):
                    transaction.append((item, itemset_1[item]))
                else:
                    print('removed:', item)
            transaction.sort(key=lambda x: x[0])
            transaction.sort(key=lambda x: x[1], reverse=True)
            print(transaction)
            root_aux = root
            for item in transaction:
                root_aux = insert_ppc_node(root_aux, item[0])
    pre_post(root)
    return root


tree = build_ppc_tree(input_file, itemset_1)

removed: 0
removed: 14
removed: 60
removed: 66
removed: 75
removed: 84
removed: 125
removed: 155
removed: 161
removed: 163
removed: 168
removed: 198
removed: 205
removed: 277
removed: 290
removed: 324
removed: 2297
removed: 2300
removed: 2354
removed: 2402
removed: 3403
removed: 4413
removed: 4414
removed: 4440
removed: 4493
removed: 4496
removed: 4499
removed: 4502
removed: 4503
removed: 4518
removed: 4525
removed: 4567
removed: 4680
removed: 4780
removed: 4786
removed: 4799
removed: 4815
removed: 4853
removed: 4933
removed: 4937
removed: 4946
removed: 5113
removed: 6465
removed: 6857
removed: 6867
removed: 6921
removed: 6962
removed: 7026
removed: 7036
removed: 7046
removed: 7057
removed: 7075
removed: 7082
[('170', 998), ('4438', 996), ('4426', 992), ('4428', 992), ('180', 989), ('184', 989), ('7062', 986), ('4432', 985), ('188', 981), ('4434', 977), ('7092', 975), ('4940', 971), ('4430', 962), ('4436', 961), ('7112', 943), ('7102', 938), ('3404', 935), ('4404', 935), ('17', 913), (

removed: 7035
removed: 7045
removed: 7057
removed: 7072
removed: 7085
[('170', 998), ('4438', 996), ('4426', 992), ('4428', 992), ('180', 989), ('184', 989), ('7062', 986), ('4432', 985), ('188', 981), ('4434', 977), ('7092', 975), ('4940', 971), ('4430', 962), ('4436', 961), ('7112', 943), ('7102', 938), ('3404', 935), ('4404', 935), ('17', 913), ('260', 904), ('265', 904)]
removed: 0
removed: 14
removed: 59
removed: 66
removed: 73
removed: 84
removed: 125
removed: 155
removed: 161
removed: 163
removed: 168
removed: 197
removed: 205
removed: 277
removed: 290
removed: 348
removed: 1320
removed: 2300
removed: 2331
removed: 2402
removed: 3403
removed: 4413
removed: 4414
removed: 4440
removed: 4493
removed: 4496
removed: 4499
removed: 4502
removed: 4503
removed: 4518
removed: 4525
removed: 4572
removed: 4680
removed: 4780
removed: 4786
removed: 4799
removed: 4814
removed: 4848
removed: 4933
removed: 4937
removed: 4946
removed: 5013
removed: 6504
removed: 6857
removed: 6867
removed: 6921
r

removed: 4440
removed: 4493
removed: 4496
removed: 4499
removed: 4502
removed: 4503
removed: 4518
removed: 4525
removed: 4567
removed: 4680
removed: 4780
removed: 4786
removed: 4800
removed: 4808
removed: 4843
removed: 4933
removed: 4937
removed: 4946
removed: 5563
removed: 6820
removed: 6857
removed: 6868
removed: 6869
removed: 6922
removed: 7022
removed: 7032
removed: 7042
removed: 7052
removed: 7072
removed: 7082
[('170', 998), ('4438', 996), ('4426', 992), ('4428', 992), ('180', 989), ('184', 989), ('7062', 986), ('4432', 985), ('188', 981), ('4434', 977), ('7092', 975), ('4940', 971), ('4430', 962), ('4436', 961), ('7112', 943), ('7102', 938), ('3404', 935), ('4404', 935), ('17', 913), ('260', 904), ('265', 904)]
removed: 2
removed: 15
removed: 56
removed: 70
removed: 75
removed: 89
removed: 111
removed: 154
removed: 162
removed: 167
removed: 169
removed: 197
removed: 252
removed: 278
removed: 284
removed: 330
removed: 1387
removed: 2300
removed: 2354
removed: 2402
removed: 3403
r

removed: 2401
removed: 3403
removed: 4413
removed: 4414
removed: 4440
removed: 4493
removed: 4496
removed: 4499
removed: 4502
removed: 4505
removed: 4523
removed: 4526
removed: 4527
removed: 4627
removed: 4727
removed: 4785
removed: 4798
removed: 4807
removed: 4833
removed: 4936
removed: 4939
removed: 4951
removed: 4953
removed: 5946
removed: 6856
removed: 6868
removed: 6869
removed: 6922
removed: 7022
removed: 7035
removed: 7042
removed: 7057
removed: 7073
removed: 7085
[('170', 998), ('4438', 996), ('4426', 992), ('4428', 992), ('180', 989), ('184', 989), ('7062', 986), ('4432', 985), ('188', 981), ('4434', 977), ('7092', 975), ('4940', 971), ('4430', 962), ('4436', 961), ('7112', 943), ('7102', 938), ('3404', 935), ('4404', 935), ('17', 913), ('260', 904), ('265', 904)]
removed: 0
removed: 14
removed: 60
removed: 68
removed: 75
removed: 84
removed: 111
removed: 158
removed: 161
removed: 163
removed: 168
removed: 197
removed: 252
removed: 277
removed: 291
removed: 387
removed: 2297
r

removed: 66
removed: 74
removed: 84
removed: 111
removed: 155
removed: 161
removed: 167
removed: 168
removed: 198
removed: 252
removed: 277
removed: 290
removed: 1227
removed: 2218
removed: 2300
removed: 2354
removed: 2402
removed: 3403
removed: 4413
removed: 4414
removed: 4440
removed: 4493
removed: 4496
removed: 4499
removed: 4502
removed: 4506
removed: 4518
removed: 4525
removed: 4559
removed: 4680
removed: 4780
removed: 4786
removed: 4799
removed: 4816
removed: 4848
removed: 4933
removed: 4937
removed: 4946
removed: 5644
removed: 6325
removed: 6857
removed: 6867
removed: 6919
removed: 6962
removed: 7026
removed: 7036
removed: 7046
removed: 7056
removed: 7074
removed: 7082
[('170', 998), ('4438', 996), ('4426', 992), ('4428', 992), ('180', 989), ('184', 989), ('7062', 986), ('4432', 985), ('188', 981), ('4434', 977), ('7092', 975), ('4940', 971), ('4430', 962), ('4436', 961), ('7112', 943), ('7102', 938), ('3404', 935), ('4404', 935), ('17', 913), ('260', 904), ('265', 904)]
removed

...

removed: 4933
removed: 4937
removed: 4946
removed: 5785
removed: 6041
removed: 6857
removed: 6867
removed: 6921
removed: 6954
removed: 7026
removed: 7036
removed: 7046
removed: 7057
removed: 7072
removed: 7082
[('170', 998), ('4438', 996), ('4426', 992), ('4428', 992), ('180', 989), ('184', 989), ('7062', 986), ('4432', 985), ('188', 981), ('4434', 977), ('7092', 975), ('4940', 971), ('4430', 962), ('4436', 961), ('7112', 943), ('7102', 938), ('3404', 935), ('4404', 935), ('17', 913), ('260', 904), ('265', 904)]
removed: 2
removed: 14
removed: 57
removed: 70
removed: 77
removed: 86
removed: 111
removed: 160
removed: 162
removed: 163
removed: 169
removed: 197
removed: 252
removed: 278
removed: 287
removed: 380
removed: 2297
removed: 2299
removed: 2301
removed: 2401
removed: 3403
removed: 4413
removed: 4414
removed: 4440
removed: 4493
removed: 4496
removed: 4499
removed: 4502
removed: 4503
removed: 4523
removed: 4526
removed: 4527
removed: 4627
removed: 4727
removed: 4785
removed: 4798
r

removed: 7046
removed: 7057
removed: 7072
removed: 7082
[('170', 998), ('4438', 996), ('4426', 992), ('4428', 992), ('180', 989), ('184', 989), ('7062', 986), ('4432', 985), ('188', 981), ('4434', 977), ('7092', 975), ('4940', 971), ('4430', 962), ('7112', 943), ('7102', 938), ('3404', 935), ('4404', 935), ('17', 913), ('260', 904), ('265', 904)]
removed: 1
removed: 15
removed: 62
removed: 66
removed: 76
removed: 84
removed: 123
removed: 155
removed: 161
removed: 167
removed: 168
removed: 197
removed: 218
removed: 277
removed: 290
removed: 348
removed: 2297
removed: 2299
removed: 2301
removed: 2401
removed: 3403
removed: 4413
removed: 4414
removed: 4440
removed: 4493
removed: 4496
removed: 4499
removed: 4501
removed: 4507
removed: 4523
removed: 4526
removed: 4527
removed: 4627
removed: 4727
removed: 4785
removed: 4798
removed: 4807
removed: 4833
removed: 4936
removed: 4939
removed: 4950
removed: 4953
removed: 5946
removed: 6856
removed: 6868
removed: 6869
removed: 6922
removed: 7022
re

removed: 4950
removed: 4953
removed: 5946
removed: 6856
removed: 6868
removed: 6869
removed: 6922
removed: 7022
removed: 7036
removed: 7042
removed: 7057
removed: 7072
removed: 7086
removed: 7105
[('170', 998), ('4438', 996), ('4426', 992), ('4428', 992), ('180', 989), ('184', 989), ('7062', 986), ('4432', 985), ('188', 981), ('4434', 977), ('7092', 975), ('4940', 971), ('4430', 962), ('4436', 961), ('7112', 943), ('3404', 935), ('4404', 935), ('17', 913), ('260', 904), ('265', 904)]
removed: 1
removed: 15
removed: 63
removed: 66
removed: 76
removed: 84
removed: 122
removed: 155
removed: 161
removed: 167
removed: 168
removed: 197
removed: 233
removed: 277
removed: 290
removed: 423
removed: 2297
removed: 2299
removed: 2301
removed: 2401
removed: 3403
removed: 4413
removed: 4414
removed: 4440
removed: 4492
removed: 4496
removed: 4499
removed: 4502
removed: 4504
removed: 4523
removed: 4526
removed: 4527
removed: 4627
removed: 4727
removed: 4785
removed: 4798
removed: 4807
removed: 4833
re

removed: 4946
removed: 5225
removed: 6835
removed: 6857
removed: 6867
removed: 6921
removed: 6962
removed: 7026
removed: 7036
removed: 7046
removed: 7057
removed: 7075
removed: 7082
[('170', 998), ('4438', 996), ('4426', 992), ('4428', 992), ('180', 989), ('184', 989), ('7062', 986), ('188', 981), ('4434', 977), ('7092', 975), ('4940', 971), ('4436', 961), ('7112', 943), ('7102', 938), ('3404', 935), ('4404', 935), ('17', 913), ('260', 904), ('265', 904)]
removed: 1
removed: 15
removed: 60
removed: 66
removed: 75
removed: 84
removed: 111
removed: 155
removed: 161
removed: 165
removed: 168
removed: 197
removed: 252
removed: 278
removed: 293
removed: 380
removed: 1330
removed: 2299
removed: 2301
removed: 2401
removed: 3403
removed: 4413
removed: 4414
removed: 4440
removed: 4493
removed: 4496
removed: 4499
removed: 4502
removed: 4506
removed: 4518
removed: 4525
removed: 4567
removed: 4680
removed: 4780
removed: 4786
removed: 4799
removed: 4815
removed: 4848
removed: 4933
removed: 4937
rem

removed: 4494
removed: 4497
removed: 4500
removed: 4503
removed: 4517
removed: 4524
removed: 4527
removed: 4627
removed: 4727
removed: 4785
removed: 4798
removed: 4807
removed: 4833
removed: 4933
removed: 4937
removed: 4945
removed: 4953
removed: 5946
removed: 6856
removed: 6866
removed: 6869
removed: 6922
removed: 7022
removed: 7032
removed: 7042
removed: 7052
removed: 7072
removed: 7082
[('170', 998), ('4438', 996), ('4426', 992), ('4428', 992), ('180', 989), ('184', 989), ('7062', 986), ('4432', 985), ('188', 981), ('4434', 977), ('7092', 975), ('4940', 971), ('4430', 962), ('4436', 961), ('7112', 943), ('7102', 938), ('3404', 935), ('4404', 935), ('17', 913), ('260', 904), ('265', 904)]
removed: 2
removed: 15
removed: 56
removed: 70
removed: 76
removed: 89
removed: 125
removed: 154
removed: 162
removed: 167
removed: 169
removed: 197
removed: 205
removed: 278
removed: 283
removed: 1297
removed: 2297
removed: 2300
removed: 2354
removed: 2402
removed: 3403
removed: 4409
removed: 4414


In [9]:
def print_tree(root):
    print(root.label, root.pre_order, root.post_order)
    if root.child:
        print_tree(root.child)
    if root.sibling:
        print_tree(root.sibling)

In [10]:
def make_n_list(root, n_list={}):
    if root.label:
        if root.label not in n_list:
            n_list[root.label] = []
        n_list[root.label].append(((root.pre_order, root.post_order), root.count))
    if root.child:
        make_n_list(root.child, n_list)
    if root.sibling:
        make_n_list(root.sibling, n_list)

In [11]:
n_list = {}
make_n_list(tree, n_list)
n_list

{'17': [((19, 2), 647),
  ((24, 9), 20),
  ((31, 15), 27),
  ((34, 20), 3),
  ((40, 24), 32),
  ((47, 32), 1),
  ((54, 38), 12),
  ((59, 44), 14),
  ((62, 47), 1),
  ((66, 51), 3),
  ((76, 60), 27),
  ((82, 67), 3),
  ((91, 75), 20),
  ((100, 85), 1),
  ((106, 91), 4),
  ((122, 107), 4),
  ((129, 113), 10),
  ((134, 120), 1),
  ((139, 125), 1),
  ((146, 131), 1),
  ((151, 137), 1),
  ((156, 143), 1),
  ((175, 159), 12),
  ((182, 167), 1),
  ((190, 175), 1),
  ((197, 182), 1),
  ((207, 192), 1),
  ((221, 205), 5),
  ((226, 212), 6),
  ((235, 222), 1),
  ((247, 232), 2),
  ((255, 239), 4),
  ((261, 246), 1),
  ((269, 254), 3),
  ((275, 261), 1),
  ((286, 271), 1),
  ((292, 279), 1),
  ((305, 290), 1),
  ((313, 297), 11),
  ((316, 302), 1),
  ((327, 312), 1),
  ((344, 329), 6),
  ((354, 340), 1),
  ((382, 366), 3),
  ((393, 378), 1),
  ((409, 393), 4),
  ((424, 411), 1),
  ((430, 417), 1),
  ((439, 424), 2),
  ((457, 442), 2),
  ((463, 447), 1),
  ((472, 459), 1),
  ((492, 476), 1)],
 '17

In [12]:
F = list(itemset_1.items())
F.sort(key=lambda x: x[0], reverse=True)
F.sort(key=lambda x: x[1])
items_ordered = [x[0] for x in F]
items_ordered

['265',
 '260',
 '17',
 '4404',
 '3404',
 '7102',
 '7112',
 '4436',
 '4430',
 '4940',
 '7092',
 '4434',
 '188',
 '4432',
 '7062',
 '184',
 '180',
 '4428',
 '4426',
 '4438',
 '170']

In [13]:
F

[('265', 904),
 ('260', 904),
 ('17', 913),
 ('4404', 935),
 ('3404', 935),
 ('7102', 938),
 ('7112', 943),
 ('4436', 961),
 ('4430', 962),
 ('4940', 971),
 ('7092', 975),
 ('4434', 977),
 ('188', 981),
 ('4432', 985),
 ('7062', 986),
 ('184', 989),
 ('180', 989),
 ('4428', 992),
 ('4426', 992),
 ('4438', 996),
 ('170', 998)]

In [14]:
def NL_interserction(n_list1, n_list2, minsup):
    n_list_result = []
    for k in n_list1:
        for l in n_list2:
            if k[0][0] < l[0][0] and k[0][1] > l[0][1]:                 
                n_list_result.append((k[0], l[1]))
    d = {x:0 for x, _ in n_list_result} 
    for name, num in n_list_result: d[name] += num 
    n_list_result = list(map(tuple, d.items()))
    return n_list_result

In [15]:
def find_subsets(items, n):
    return list(itertools.combinations(items, n))


def get_all_subsets(items):
    subsets = [find_subsets(items, i) for i in range(len(items)+1)]
    subsets = ['-'.join(map(str, item)) for item in list(itertools.chain.from_iterable(subsets))]
    return subsets[1:]

In [16]:
def get_n_list(key):
    global n_list
    return n_list[key] if key in n_list else []

In [17]:
def print_nodes(nodes):
    print("Nodes:")
    for node in nodes:
        print(node.label)
    print("\n")

In [18]:
def building_pattern_tree(cur_no, next_nos, father_no=None):
    global support
    global n_list
    global F
    
    if cur_no.equivalent_items is None:
        cur_no.equivalent_items = []
        
    cur_no.child_nodes = []
    if father_no is not None:
        p1 = get_n_list('-'.join([father_no.label, cur_no.label]))
    else:
        p1 = get_n_list(cur_no.label)
        
    for i in next_nos:
        if father_no is not None:
            p2 = get_n_list('-'.join([father_no.label, i.label]))
        else:
            p2 = get_n_list(i.label)
            
        p = NL_interserction(p2, p1, support)
        p_support = sum([item[1] for item in p])
        
        if p_support == cur_no.support:
            cur_no.equivalent_items += [i.label]
        elif p_support >= support:
            child = FPTNode()
            child.label = i.label
            child.support = p_support
            cur_no.child_nodes += [child]
            if father_no is not None:
                n_list['-'.join([father_no.label, cur_no.label, child.label])] = p
            else:
                n_list['-'.join([cur_no.label, child.label])] = p

    if father_no is not None:
        cur_no.label = '-'.join([father_no.label, cur_no.label])
        
    F.append((cur_no.label, cur_no.support))
    
    if len(cur_no.equivalent_items) > 0:
        subsets = get_all_subsets(cur_no.equivalent_items)
        cand_itemsets = [('-'.join([cur_no.label, item]), cur_no.support) for item in subsets]
        F += cand_itemsets
    
    if len(cur_no.child_nodes) > 0:
        for child in cur_no.child_nodes:
            aheads_ = [i for i in cur_no.child_nodes[cur_no.child_nodes.index(child)+1:]]
            child.equivalent_items = list(cur_no.equivalent_items)
            building_pattern_tree(child, aheads_, cur_no)

In [19]:
nodes = []
for key in list(items_ordered):
    node = FPTNode()
    node.label = key
    node.support = sum([item[1] for item in n_list[key]])
    nodes.append(node)

In [20]:
F = []
for node in list(nodes):
    aheads = nodes[nodes.index(node)+1:]
    building_pattern_tree(node, aheads)

In [21]:
F

[('265', 904),
 ('265-260', 904),
 ('265-4438', 900),
 ('265-4438-260', 900),
 ('265-170', 902),
 ('265-170-260', 902),
 ('260', 904),
 ('260-4438', 900),
 ('260-170', 902),
 ('17', 913),
 ('17-188', 901),
 ('17-188-170', 900),
 ('17-184', 906),
 ('17-184-180', 906),
 ('17-184-4438', 902),
 ('17-184-4438-180', 902),
 ('17-184-4438-170', 901),
 ('17-184-4438-170-180', 901),
 ('17-184-170', 905),
 ('17-184-170-180', 905),
 ('17-180', 906),
 ('17-180-4438', 902),
 ('17-180-4438-170', 901),
 ('17-180-170', 905),
 ('17-4428', 905),
 ('17-4428-4426', 901),
 ('17-4428-4426-170', 900),
 ('17-4428-4438', 901),
 ('17-4428-4438-170', 900),
 ('17-4428-170', 904),
 ('17-4426', 905),
 ('17-4426-4438', 901),
 ('17-4426-4438-170', 900),
 ('17-4426-170', 904),
 ('17-4438', 909),
 ('17-4438-170', 908),
 ('17-170', 912),
 ('4404', 935),
 ('4404-3404', 935),
 ('4404-4940', 907),
 ('4404-4940-3404', 907),
 ('4404-4940-4438', 903),
 ('4404-4940-4438-3404', 903),
 ('4404-4940-4438-170', 901),
 ('4404-4940-44

In [22]:
import json
with open('outfile', 'w') as f__:
    f__.write(json.dumps(F))